# Random Sampling in a Hyperplane Arrangement Complement

### Import Packages

In [93]:
## only run this once
using LinearAlgebra
include("lambda_cyclic.jl")
include("hyperplanes.jl")
include("sampling.jl")

getNeighbors (generic function with 1 method)

### Generate a random point

In [2]:
p(n) = [rand(Float64)-.5 for j=1:n]

p (generic function with 1 method)

### Generate the 3-Sunlet hyperplanes

In [6]:
coords_3 = ["mu_0", "mu_1", "mu_2", "eta_{10}", "eta_{21}", "eta_{02}"]
coords_3[[i for i in eachindex(H3[1]) if H3[1][i] != 0]]

1-element Vector{String}:
 "mu_0"

In [8]:
coords_4 = ["mu_0", "mu_1", "mu_2", "mu_3", "eta_{10}", "eta_{21}", "eta_{32}", "eta_{03}"]

8-element Vector{String}:
 "mu_0"
 "mu_1"
 "mu_2"
 "mu_3"
 "eta_{10}"
 "eta_{21}"
 "eta_{32}"
 "eta_{03}"

In [3]:
Z5 = FiniteCyclicGroup([5])
H5 = generateSunletArr(Z5)
sort(H5)

21-element Vector{Any}:
 [0, 0, 0, 0, 1, -1, -1, -1, -1]
 [0, 0, 0, 0, 1, 0, 0, 0, 1]
 [0, 0, 0, 0, 1, 0, 0, 1, 0]
 [0, 0, 0, 0, 1, 0, 1, 0, 0]
 [0, 0, 0, 0, 1, 1, 0, 0, 0]
 [0, 0, 0, 1, 0, -1, -1, -1, 0]
 [0, 0, 0, 1, 0, 0, -1, -1, -1]
 [0, 0, 0, 1, 0, 0, 0, 1, 1]
 [0, 0, 0, 1, 0, 0, 1, 1, 0]
 [0, 0, 0, 1, 0, 1, 1, 0, 0]
 ⋮
 [0, 0, 1, 0, 0, 0, 0, -1, -1]
 [0, 0, 1, 0, 0, 0, 1, 1, 1]
 [0, 0, 1, 0, 0, 1, 1, 1, 0]
 [0, 1, 0, 0, 0, -1, 0, 0, 0]
 [0, 1, 0, 0, 0, 0, -1, 0, 0]
 [0, 1, 0, 0, 0, 0, 0, -1, 0]
 [0, 1, 0, 0, 0, 0, 0, 0, -1]
 [0, 1, 0, 0, 0, 1, 1, 1, 1]
 [1, 0, 0, 0, 0, 0, 0, 0, 0]

### Inequality vector

In [5]:
ineq(v, H) = [LinearAlgebra.dot(v, h) > 0 for h in H]

ineq (generic function with 1 method)

In [6]:
neighbors(I, J) = (sum(I .⊻ J) == 1)

neighbors (generic function with 1 method)

### Sampling Algorithm

## Example $n = 3$

In [74]:
H3

7-element Vector{Any}:
 [0, 0, 1, 1, 0]
 [0, 0, 1, -1, -1]
 [0, 1, 0, 1, 1]
 [0, 1, 0, 0, -1]
 [0, 1, 0, -1, 0]
 [0, 0, 1, 0, 1]
 [1, 0, 0, 0, 0]

In [92]:
Z3 = FiniteCyclicGroup([3])
H3 = generateSunletArr(Z3)
@time s3 = sample(Z3, H3, 2^20); ## get the sample points (< 2 seconds usually)
println(length(s3))
ranks = countRanks(s3, Z3)
ranks

  2.184808 seconds (11.69 M allocations: 327.922 MiB, 4.44% gc time, 8.01% compilation time)
92


Dict{Int64, Int64} with 3 entries:
  7 => 4
  9 => 64
  8 => 24

In [98]:
## getting poset structure!
## start with a parochial chamber, say [0 0 ... 0]
## find all neighbors, add to list of distance 1 chambers
## record the distance 1 chambers as visited already
## for each distance 1 chamber, compute the neighbors
## if the neighbor hasn't been visited yet, add it to the distance 2 chambers
## add the distance 2 chambers to the visited chambers
## etc


Dict(1 => [0, 0,

 0, 0, 0, 0, 0])

TypeError: TypeError: non-boolean (Vector{Vector{Bool}}) used in boolean context

## checking if $\mu_0$ does anything...

In [19]:
inequalityKeys = collect(keys(s3))
for I in inequalityKeys
    for J in inequalityKeys
#         println(I .⊻ J)
        if I .⊻ J == [Bool(1), Bool(0), Bool(0), Bool(0), Bool(0), Bool(0), Bool(0)]
            print(getRank(s3[I], [3]) - getRank(s3[J], [3]))
        end
    end
end

In [20]:
print([Bool(1), Bool(0), Bool(0), Bool(0), Bool(0), Bool(0)])

Bool[1, 0, 0, 0, 0, 0]

### Test Neighbors

In [21]:
for I in collect(keys(s3))
    s = s3[I]
    curr_rank = getRank(s, n)
    for J in collect(keys(s3))
        if neighbors(I, J)
            neighbor = s3[J]
            n_rank = getRank(neighbor, n)
            print(abs(curr_rank - n_rank))
        end
    end
end

MethodError: MethodError: no method matching getRank(::Vector{Float64}, ::Int64)
Closest candidates are:
  getRank(::Any, !Matched::FiniteCyclicGroup) at c:\Users\spcox\Documents\GitHub\3-Sunlet\lambda_cyclic.jl:129

### Parochial Chambers

In [22]:
p1 = s3[[1, 1, 1, 1, 1, 1, 1]];
p2 = s3[[0, 0, 0, 0, 0, 0, 0]];
r1 = getRank(p1, n);
r2 = getRank(p2, n);
println(r1, r2)
parochial(s3, 7)
# keys(s3)

KeyError: KeyError: key [0, 0, 0, 0, 0, 0, 0] not found

In [23]:
function parochial(S, k)
    p0 = S[zeros(Bool, k)]
    p1 = S[ones(Bool, k)]
    r0 = getRank(p0, n)
    r1 = getRank(p1, n)
    print(r0, ' ', r1)
end

parochial (generic function with 1 method)

In [24]:
p_avg = (p1 .+ p2)./2; ## average the sample representing parochial chambers
r_avg = getRank(p_avg, n) ## the average happens to have full rank

UndefVarError: UndefVarError: p2 not defined

### Low Rank Chambers

In [25]:
low_rank = []
s3_keys = collect(keys(s3))
for s in eachindex(s3_keys)
    if ranks[s] == 7
        push!(low_rank, s)
    end
end
length(low_rank)

UndefVarError: UndefVarError: ranks not defined

In [26]:
## two of the chambers are parochial
s3_keys[low_rank]

Any[]

### Testing to_lambda function

In [27]:
s0 = collect(values(s3))[1];
ineq(s0, H3)

7-element Vector{Bool}:
 1
 1
 1
 1
 1
 1
 1

In [28]:
G = [[0, 2, 1], [1, 1, 1], [1, 0, 2], [1, 2, 0], [2, 2, 2], [2, 1, 0], [2, 0, 1]]
for g in G
    print(computeLambda(g, m1, to_lambda(s0[1:3], s0[4:6], 3), 3), ' ')
    println(computeLambda(g, m2, to_lambda(s0[1:3], s0[4:6], 3), 3))
end

BoundsError: BoundsError: attempt to access 5-element Vector{Float64} at index [4:6]

In [29]:
s0

5-element Vector{Float64}:
 0.31449080493938786
 0.9376601187603265
 0.6892820277376034
 0.08449822373118632
 0.544580190470173

## Example $n = 4$

### Generate the Hyperplanes

In [30]:
## mu 0 1 2 3 eta 10 21 32 03
H4 = [[1 0 0 0 0 0 0 0], [0 0 1 0 0 -1 -1 0], [0 0 1 0 -1 -1 0 0], [0 0 1 0 0 1 1 0], [0 0 1 0 1 1 0 0], [0 1 0 0 0 0 -1 0], [0 1 0 0 0 -1 0 0], [0 1 0 0 -1 0 0 0], [0 1 0 0 0 0 0 -1], [0 0 0 1 0 0 0 1], [0 0 0 1 0 0 1 0], [0 0 0 1 0 1 0 0], [0 0 0 1 1 0 0 0]]

13-element Vector{Matrix{Int64}}:
 [1 0 … 0 0]
 [0 0 … -1 0]
 [0 0 … 0 0]
 [0 0 … 1 0]
 [0 0 … 0 0]
 [0 1 … -1 0]
 [0 1 … 0 0]
 [0 1 … 0 0]
 [0 1 … 0 -1]
 [0 0 … 0 1]
 [0 0 … 1 0]
 [0 0 … 0 0]
 [0 0 … 0 0]

### Generate Sample Points

In [31]:
## 2328

In [32]:
function sample_info(n, H, N)
    @time S = sample(n, H, N)
    println(length(S))
    ranks = [getRank(s, n) for s in collect(values(S))];
    R = sort(collect(Set(ranks)))
    println("R=", R)
    counts = [sum([r==k for r in ranks]) for k in R]; ## tabulate the ranks
    println(counts)
    priors = [100*c/sum(counts) for c in counts]; ## compute priors (% of samples with given rank)
    println(priors)
    return S
end

sample_info (generic function with 1 method)

In [33]:
Z4 = FiniteCyclicGroup([4])
@time S = sample(4, H4, 2^20)
println(length(S))
ranks = [getRank(s, Z4) for s in collect(values(S))];
R = sort(collect(Set(ranks)))
println("R=", R)
counts = [sum([r==k for r in ranks]) for k in R]; ## tabulate the ranks
println(counts)
priors = [100*c/sum(counts) for c in counts]; ## compute priors (% of samples with given rank)
println(priors)

MethodError: MethodError: no method matching sample(::Int64, ::Vector{Matrix{Int64}}, ::Int64)
Closest candidates are:
  sample(!Matched::FiniteCyclicGroup, ::Any, ::Int64) at c:\Users\spcox\Documents\GitHub\3-Sunlet\sampling.jl:1

In [34]:
IByRank4 = Dict([r=>[s for s in collect(keys(S)) if getRank(S[s], Z4) == r] for r in R])

UndefVarError: UndefVarError: R not defined

In [35]:
start_chamber = IByRank4[10][1]

UndefVarError: UndefVarError: IByRank4 not defined

In [36]:
inequalityKeys = collect(keys(s4))
for I in inequalityKeys
    for J in inequalityKeys
#         println(I .⊻ J)
        if I .⊻ J == [Bool(1), Bool(0), Bool(0), Bool(0), Bool(0), Bool(0), Bool(0), Bool(0), Bool(0), Bool(0), Bool(0), Bool(0), Bool(0)]
            print(getRank(s4[I], [3]) - getRank(s4[J], [3]))
        end
    end
end

UndefVarError: UndefVarError: s4 not defined

### Check Neighbors

In [37]:
function getNeighbors(I, allI)
    neighborhood = []
    for J in allI
        if neighbors(I, J)
            push!(neighborhood, J)
        end
    end
    return neighborhood
end

getNeighbors (generic function with 1 method)

In [38]:
function check_neighbors(S, group)
    allI = collect(keys(S))
    
    for I in allI
        curr_rank = getRank(s, group)
        neighbor_ranks = getNeighborRanks(I, S, group)
        high_neighbor_ranks = neighbor_ranks[i for i in eachindex(neighbor_ranks) if abs(neighbor_ranks[i] - curr_rank) > 1]
        println(curr_rank, high_neighbor_ranks)
    end
end

check_neighbors (generic function with 1 method)

In [39]:
function getNeighborRanks(I, S, group)
    neighbor_ranks = []
    for J in getNeighbors(I, collect(keys(S)))
        push!(neighbor_ranks, getRank(S[J], group))
    end
#     return [getRank(S[J], group) for J in getNeighbors(I, collect(keys(S)))
    return neighbor_ranks
end

getNeighborRanks (generic function with 1 method)

In [40]:
function moveUp(curr_chamber, all_chambers, group)
    curr_rank = getRank(all_chambers[curr_chamber], group)
    neighbor_ranks = getNeighborRanks(curr_chamber, all_chambers, group)
    next_chamber = getNeighbors(curr_chamber, collect(keys(all_chambers)))[findfirst(x -> x > curr_rank, neighbor_ranks)]
    return next_chamber
end

moveUp (generic function with 1 method)

In [41]:
rk11_chamber = moveUp(start_chamber, S, Z4)
rk12_chamber = moveUp(rk11_chamber, S, Z4)
rk13_chamber = moveUp(rk12_chamber, S, Z4)
rk14_chamber = moveUp(rk13_chamber, S, Z4)
rk15_chamber = moveUp(rk14_chamber, S, Z4)
rk16_chamber = moveUp(rk15_chamber, S, Z4)

UndefVarError: UndefVarError: start_chamber not defined

### Check Parochial Chambers

In [42]:
p0 = s4[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]];
p1 = s4[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]];
r0 = getRank(p0, 4)
r1 = getRank(p1, 4)
print(r0, ' ', r1)

UndefVarError: UndefVarError: s4 not defined

### Low Rank Chambers

In [43]:
low_rank_4 = []
for s in collect(keys(s4))
    if getRank(s4[s], 4) == 10
        push!(low_rank_4, s)
    end
end
length(low_rank)

UndefVarError: UndefVarError: s4 not defined

In [44]:
low_rank_4

Any[]

### Dictionary of ranks => inequalities

In [45]:
rdict_3 = Dict([r => [s3_keys[s] for s in eachindex(collect(keys(s3))) if ranks[s] == r] for r in R])

UndefVarError: UndefVarError: R not defined

In [46]:
length(rdict_3[9])

UndefVarError: UndefVarError: rdict_3 not defined

In [47]:
function rankToIDict(S, R)
    rToI = Dict()
    K = collect(keys(S))
    for r in Set(R)
        rToI[r] = [K[s] for s in eachindex(K) if R[s] == r]
    end
    return rToI
end

rankToIDict (generic function with 1 method)

In [48]:
rdict_4 = rankToIDict(s4, [getRank(s, 4) for s in collect(values(s4))])

UndefVarError: UndefVarError: s4 not defined

In [49]:
for r in collect(keys(rdict_4))
    println(length(rdict_4[r]))
end

UndefVarError: UndefVarError: rdict_4 not defined

In [50]:
function printRanks(RI, file_name)
    open(file_name, "w") do io
        for r in sort(collect(keys(RI)))
            println(io, "## rank = ", r)

            count = 0
            for I in RI[r]
                if count % 5 == 0
                    println(io, "")
                end
                for b in I
                    print(io, Int(b), " ")
                end
                println(io, "")
                count += 1
            end

            println(io, "")
        end
    end
end

printRanks (generic function with 1 method)

In [51]:
function latexRanks(RI)
    count = 1
    for r in sort(collect(keys(RI)))
        for I in RI[r]
            print(count, " & ")
            for i in eachindex(I)
                if i < length(I)
                    print(Int(I[i]), " & ")
                else
                    print(Int(I[i]), " & ", r, " & \\\\")
                end
            end
            println()
            count += 1
        end
    end
end

latexRanks (generic function with 1 method)

In [52]:
latexRanks(rdict_3);

UndefVarError: UndefVarError: rdict_3 not defined

## Z2 x Z2

In [53]:
## mu 0 1 2 3, eta 10 21 32 03
H22 = [[1 0 0 0 0 0 0 0], [0 1 0 0 1 0 0 0], [0 1 0 0 0 0 1 0], [0 1 0 0 -1 0 0 0], [0 1 0 0 0 0 -1 0], [0 0 1 0 1 1 0 0], [0 0 1 0 0 1 1 0], [0 0 1 0 -1 -1 0 0], [0 0 1 0 0 -1 -1 0], [0 0 0 1 0 0 0 1], [0 0 0 1 0 1 0 0], [0 0 0 1 0 0 0 -1], [0 0 0 1 0 -1 0 0]]

13-element Vector{Matrix{Int64}}:
 [1 0 … 0 0]
 [0 1 … 0 0]
 [0 1 … 1 0]
 [0 1 … 0 0]
 [0 1 … -1 0]
 [0 0 … 0 0]
 [0 0 … 1 0]
 [0 0 … 0 0]
 [0 0 … -1 0]
 [0 0 … 0 1]
 [0 0 … 0 0]
 [0 0 … 0 -1]
 [0 0 … 0 0]

In [54]:
# HToLatex(H22, getValidGroupTriples(Z2Z2), coords_4)
function getGroupIdentity(group)
    return zeros(Int64, getNumFactors(group))
end

validTriples = getValidGroupTriples(Z2Z2)
nonZeroTriples = [G for G in validTriples if G[1] != getGroupIdentity(Z2Z2)]

UndefVarError: UndefVarError: Z2Z2 not defined

In [55]:
validTriples[1][1] == getGroupIdentity(Z2Z2)

UndefVarError: UndefVarError: validTriples not defined

In [56]:
groupStructure = [2,2]
Z2Z2 = FiniteCyclicGroup([2,2])
@time s22 = sample(4, H22, 2^20); ## get the sample points (< 2 seconds usually)
println(length(s22))

MethodError: MethodError: no method matching sample(::Int64, ::Vector{Matrix{Int64}}, ::Int64)
Closest candidates are:
  sample(!Matched::FiniteCyclicGroup, ::Any, ::Int64) at c:\Users\spcox\Documents\GitHub\3-Sunlet\sampling.jl:1

In [57]:
ranks = [getRank(s, Z2Z2) for s in collect(values(s22))]; ## get the rank for each sample point
R = sort(collect(Set(ranks)))
ranksByI = Dict([r=>[s for s in collect(keys(s22)) if getRank(s22[s], Z2Z2)==r] for r in R])
println("R=", R)
counts = [sum([r==k for r in ranks]) for k in R]; ## tabulate the ranks
priors = [c/sum(counts) for c in counts]; ## compute priors (% of samples with given rank)
counts

UndefVarError: UndefVarError: s22 not defined

In [58]:
start_chamber = ranksByI[10][1]

UndefVarError: UndefVarError: ranksByI not defined

In [59]:
printRanks(ranksByI, "z2z2_chamber_ranks.txt")

UndefVarError: UndefVarError: ranksByI not defined

In [60]:
priors

UndefVarError: UndefVarError: priors not defined

In [61]:
check_neighbors(s22, [2,2])

UndefVarError: UndefVarError: s22 not defined

In [62]:
HToLatex(H22, getValidGroupTriples(Z2Z2), coords_4)

UndefVarError: UndefVarError: HToLatex not defined

## Z/5Z

In [63]:
@time s5 = sample(5, H5, 2^30); ## get the sample points (~ 22 min w/ laptop set to best performance)
println(length(s5))

MethodError: MethodError: no method matching sample(::Int64, ::Vector{Any}, ::Int64)
Closest candidates are:
  sample(!Matched::FiniteCyclicGroup, ::Any, ::Int64) at c:\Users\spcox\Documents\GitHub\3-Sunlet\sampling.jl:1

In [64]:
# 113350

## changing ranks indicates an error in the hyperplane generator... :(

In [65]:
ranks = [getRank(s, Z5) for s in collect(values(s5))]; ## get the rank for each sample point
R = sort(collect(Set(ranks)))
println("R=", R)
counts = [sum([r==k for r in ranks]) for k in R]; ## tabulate the ranks
priors = [c/sum(counts) for c in counts]; ## compute priors (% of samples with given rank)
counts

UndefVarError: UndefVarError: s5 not defined

In [66]:
priors

UndefVarError: UndefVarError: priors not defined

In [67]:
check_neighbors(s5, [5])

UndefVarError: UndefVarError: s5 not defined

In [68]:
coords_5 = ["mu_0", "mu_1", "mu_2", "mu_3", "mu_4", "eta_{10}", "eta_{21}", "eta_{32}", "eta_{43}", "eta_{04}"]
HToLatex(H5, coords_5)

UndefVarError: UndefVarError: HToLatex not defined

In [69]:
inequalityKeys5 = collect(keys(s5))

UndefVarError: UndefVarError: s5 not defined

In [70]:
for i=eachindex(inequalityKeys5)
    if ranks[i] == 10
        println(inequalityKeys5)
    end
end

UndefVarError: UndefVarError: inequalityKeys5 not defined